# Model Output Data Visualization

### Read Me

This notebook vizualizes model outputs for supervised neural methods from the rPPG Toolbox. To use this notebook follow the following steps:

* STEP 1: Specify the `TEST.OUTPUT_SAVE_DIR` field in the `.yaml` config file of a neural method experiment.
* STEP 2: Run the neural method experiment. A `.pickle` file containing the test-data predictions and labels will be created in `TEST.OUTPUT_SAVE_DIR`.
* STEP 3: Add the path to the `pickle` file in the `TODO: Variable to Set` section as `data_out_path`.
* STEP 4: Add values for `trial_idx` (a value between 0 and the `Num Trials` print in the previous cell), `chunk_size` (number of samples to plot out), `chunk_num` (the chunk of size `chunk_size` in the signal). We suggest using the defaults at first.
* STEP 5: Run all cells. The predicted output, plotted against the ground truth ppg waveform, will be plotted in the final cell

### Imports

In [1]:
# !pip install natsort
!pip install scipy
# !pip install natsort
# !pip install ipywidgets

import cv2
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
import re
import glob

import torch

import numpy as np
# import ipywidgets as widgets
from IPython.display import display, clear_output
from natsort import natsorted
import os
import scipy
from scipy.sparse import spdiags
from scipy.signal import butter
import math
from scipy import linalg
from scipy import signal
from scipy import sparse

### Helper Functions

In [2]:
# HELPER FUNCTIONS

def _reform_data_from_dict(data, flatten=True):
    """Helper func for calculate metrics: reformat predictions and labels from dicts. """
    sort_data = sorted(data.items(), key=lambda x: x[0])
    sort_data = [i[1] for i in sort_data]
    sort_data = torch.cat(sort_data, dim=0)

    if flatten:
        sort_data = np.reshape(sort_data.cpu(), (-1))
    else:
        sort_data = np.array(sort_data.cpu())

    return sort_data

def _process_signal(signal, fs=30, diff_flag=True):
    # Detrend and filter
    use_bandpass = True
    if diff_flag:  # if the predictions and labels are 1st derivative of PPG signal.
        gt_bvp = _detrend(np.cumsum(signal), 100)
    else:
        gt_bvp = _detrend(signal, 100)
    if use_bandpass:
        # bandpass filter between [0.75, 2.5] Hz
        # equals [45, 150] beats per min
        [b, a] = butter(1, [0.75 / fs * 2, 2.5 / fs * 2], btype='bandpass')
        signal = scipy.signal.filtfilt(b, a, np.double(signal))
    return signal

def _detrend(input_signal, lambda_value):
    """Detrend PPG signal."""
    signal_length = input_signal.shape[0]
    # observation matrix
    H = np.identity(signal_length)
    ones = np.ones(signal_length)
    minus_twos = -2 * np.ones(signal_length)
    diags_data = np.array([ones, minus_twos, ones])
    diags_index = np.array([0, 1, 2])
    D = spdiags(diags_data, diags_index,
                (signal_length - 2), signal_length).toarray()
    detrended_signal = np.dot(
        (H - np.linalg.inv(H + (lambda_value ** 2) * np.dot(D.T, D))), input_signal)
    return detrended_signal


### TODO: Variables To Set

In [3]:
data_out_paths = glob.glob(r"../../runs/exp/*/saved_test_outputs/*.pickle")
trial_idx = 0
chunk_size = 20*30 # size of chunk to visualize: -1 will plot the entire signal
chunk_num = 0

### Read Data, and Extract Trials List

In [4]:
# Read in data and list subjects
data_list = []
for i in data_out_paths:
    with open(i, 'rb') as f:
        data_list.append(pickle.load(f))
        
    # List of all video trials
    trial_list = list(data_list[0]['predictions'].keys())
print('Num Trials', len(trial_list))

Num Trials 3


In [5]:
# Reform label and prediction vectors from multiple trial chunks
import random
from torch import randint


predictions = [np.array(_reform_data_from_dict(data['predictions'][trial_list[trial_idx]])) for data in data_list]
names = [ '_'.join(x.split('_')[:2]) for x in map(lambda x: x.split(os.sep)[-1],data_out_paths)]
names.append('Labels')
label = np.array(_reform_data_from_dict(data_list[0]['labels'][trial_list[trial_idx]]))
plt.figure(figsize=(10,5))
for data in data_list:
    # Read in meta-data from pickle file
    fs = data['fs'] # Video Frame Rate
    label_type = data['label_type'] # PPG Signal Transformation: `DiffNormalized` or `Standardized`
    diff_flag = (label_type == 'DiffNormalized')

    if chunk_size == -1:
        chunk_size = len(prediction)
        chunk_num = 0

    # Process label and prediction signals
    prediction = _process_signal(prediction, fs, diff_flag=diff_flag)
    label = _process_signal(label, fs, diff_flag=diff_flag)
    start = (chunk_num)*chunk_size
    stop = (chunk_num+1)*chunk_size
    samples = stop - start
    x_time = np.linspace(0, samples/fs, num=samples)
    plt.plot(x_time, prediction[start:stop], color=(random.random(), random.random(), random.random()))
#plt.plot(x_time, label[start:stop], color='black')
plt.title('Trial: ' + trial_list[trial_idx])
plt.legend(names)
plt.xlabel('Time (s)')
plt.show();


NameError: name 'prediction' is not defined

<Figure size 1000x500 with 0 Axes>